In [14]:
# Di file model_code/02_train_resnet.ipynb - Sel 1

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# --- Parameter Wajib (Harus Sama dengan Data Generator) ---
INPUT_SHAPE = (224, 224, 3) 
NUM_CLASSES = 1            # Klasifikasi Biner (1 neuron output)

# Di file model_code/02_train_resnet.ipynb - Sel 1 (Bagian Awal)

# --- DEFINISI VARIABEL YANG HILANG ---
EPOCHS = 30 
BATCH_SIZE = 32
# --- END DEFINISI VARIABEL ---

# 1. Muat Model Dasar ResNet50 (Transfer Learning)
base_model = ResNet50(
    input_shape=INPUT_SHAPE,
    include_top=False,  # Jangan sertakan lapisan klasifikasi ImageNet
    weights='imagenet'  # Gunakan bobot pra-latih dari ImageNet
)

# 2. Pembekuan Lapisan Dasar (FREEZE)
# Lapisan ResNet50 TIDAK akan dilatih ulang, hanya digunakan sebagai ekstraktor fitur
for layer in base_model.layers:
    layer.trainable = False

# 3. Definisikan Model Sekuensial (Lapisan Klasifikasi Baru)
model = Sequential([
    base_model,
    Flatten(),
    Dropout(0.5), # Mencegah Overfitting
    Dense(128, activation='relu'),
    # Lapisan Output BINER: 1 neuron, aktivasi sigmoid
    Dense(NUM_CLASSES, activation='sigmoid') 
])

# 4. Kompilasi Model
optimizer = Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy', # Fungsi Loss untuk 2 kelas (Biner)
    metrics=['accuracy', 
              tf.keras.metrics.Precision(), 
              tf.keras.metrics.Recall()] # Tambahkan metrik Presisi & Recall
)

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,433,025 (138.98 MB)

 Trainable params: 12,845,313 (49.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [16]:
# Di file model_code/02_train_resnet.ipynb - Sel 2

# PASTIKAN Anda menjalankan kembali SEL 5 dari notebook 01 sebelum menjalankan sel ini!
# Atau, Anda harus memuat variabel-variabel tersebut ke memori.

EPOCHS = 30 
BATCH_SIZE = 32

# Callback untuk menyimpan bobot model terbaik selama training
checkpoint = ModelCheckpoint(
    filepath='../web_app/tb_resnet_best.h5', # Simpan model di folder web_app
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True,
    mode='min'
)

# Pelatihan Model (Gunakan bobot kelas untuk mengatasi imbalance)
print(f"Jumlah sampel training: {train_generator.samples}")
print(f"Jumlah sampel validasi: {val_generator.samples}")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    class_weight=bobot_kelas_dictionary, # MENGATASI CLASS IMBALANCE
    callbacks=[checkpoint]
)

print("Pelatihan selesai. Model terbaik disimpan di tb_resnet_best.h5")

Jumlah sampel training: 0
Jumlah sampel validasi: 0


ValueError: The PyDataset has length 0